In [422]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from plotly.subplots import make_subplots
import plotly.express as px

In [423]:

def clean(series):
    '''
    A function to clean columns in a dataframe to all lowercase strings.
    Returns a cleaned list of the given column.
    '''
    cleaned = []
    for row in series:
        cleaned.append(str(row).lower())
    return cleaned

def clean_df(df):
    '''
    A function to clean columns in a dataframe to all lowercase strings,
    cleans whitespace, and replaces within-name whitespace with underscores.
    Returns a dataframe.
    '''
    df.columns = [name.strip().lower().replace(' ','_') for name in df.columns]
    return df

def reformat_census_df(df, num_levels):
    '''
    Returns a reformatted US census dataframe, with multi-indexes.
    This function only takes in US census csv data, and must meet these requirements.
    - The faux-index is named "Label (Grouping)"
    - Faux-index includes one string of a multi-index, with '!!' as the delimiter
    - Has a County and Ratio equivalent within the multi-index levels
        - Ratio ex: Exstimate, percent
    '''
    df.set_index('Label (Grouping)', inplace=True)
    df = df.transpose()
    df['temp_index'] = df.index
    regex_pattern = r"^[\w,\s]+"
    for i in range(num_levels - 1):
        regex_pattern += "\!\![\w,\s]+"
    regex_pattern += "$"
    df = df.loc[df['temp_index'].str.contains(regex_pattern)]
    df.insert(0, "County", "")
    df.insert(1, "Ratio", "")

    for row in df['temp_index']:
        terms = row.split("!!")
        terms[0] = terms[0].split(",")[0]
        df.loc[row, ['County']] = terms[0]
        df.loc[row, ['Ratio']] = terms[1]

    df.rename_axis(None, axis=1, inplace=True)
    df.set_index(['County', 'Ratio'], inplace=True)
    df.drop(columns=['temp_index'], inplace=True, axis=1)
    return df

 1. How does language diversity in each Washington county affect their educational attainment outcomes?


In [424]:
language_demographic = pd.read_csv("data/S1601LanguagesSpokenAtHome.csv")
language_demographic = reformat_census_df(language_demographic, 3)
language_demographic = clean_df(language_demographic)

enrollment = pd.read_csv("data/S1401SchoolEnrollment.csv")
enrollment = reformat_census_df(enrollment, 3)
enrollment = clean_df(enrollment)

language_demographic = language_demographic.loc[(language_demographic.index.get_level_values("County"), "Percent"), :]
enrollment = enrollment.loc[(enrollment.index.get_level_values("County"), "Percent"), :]

sub_language = language_demographic.iloc[:, [1, 2, 4, 8, 12, 16]].apply(lambda x: x.str.replace('%','').astype(float) / 100)
sub_enrollment = enrollment.iloc[:, [7, 8]].apply(lambda x: x.str.replace('%','').astype(float) / 100)

new = pd.merge(sub_enrollment, sub_language, on=['County'])

all_lang_colors = ['#AB63FA', '#FFA15A', "#FF97FF", "#EF553B"]

fig = make_subplots(2,2, subplot_titles=('Spanish Speakers',  'Other Indo-European Language Speakers', 
                                         'Asian & Pacific Island Language Speakers', 'Other Language Speakers'))

for (i, j), type, color, bool in zip([(1, 1), (1, 2), (2, 1), (2, 2)], 
                                     ["spanish", "other_indo-european_languages", "asian_and_pacific_island_languages", "other_languages"], 
                                     all_lang_colors,
                                     [True, False, False, False]):
       fig.add_trace(go.Bar(x=new.index, y=new['college,_undergraduate'],
                            name='Undergraduate',
                            showlegend=bool,
                            marker_color = 'green',
                            opacity=0.4,
                            marker_line_color='rgb(8,48,107)',
                            marker_line_width=2),
                     row = i, col = j)
       fig.add_trace(go.Bar(x=new.index, y=new['graduate,_professional_school'],
                            name='Postgraduate Enrollment', 
                            showlegend=bool,                  
                            marker_color = 'blue',                     
                            opacity=0.4,
                            marker_line_color='rgb(8,48,107)',
                            marker_line_width=2),
                     row = i, col = j)
       fig.add_trace(go.Scatter(x=new.index, y=new[type], line=dict(color=color), name=type), row = i, col = j)
       
fig.update_layout(autosize=False, width=2500, height=1250, 
                  title_text="Undergraduate & Postgraduate Enrollment Percentage Compared to Percentage of Language Speakers in Each County",
                  title_x=0.45, xaxis_title="Counties", yaxis_title="Percentage")

for i in range(1, 5):
       fig['layout'][f"xaxis{i}"]['title']='Counties'
       fig['layout'][f"yaxis{i}"]['title']='Percentage'

fig

2. how do school district budgets and spending affect spoken langugae diversity in WA counties?

In [425]:
ppe = pd.read_csv('data/SchoolDistrictPPEAverages.csv')
ppe = clean_df(ppe)
ppe['districtname'] = clean(ppe['districtname'])
counties = pd.read_csv("data/Counties.csv", header=2)
counties = counties.loc[counties["State Postal Code"] == "WA"]
counties = clean_df(counties)
counties['school_district_name'] = clean(counties['school_district_name'])

counties_ppe = ppe.merge(counties, right_on="school_district_name", left_on="districtname")
counties_ppe.set_index('school_district_name', inplace=True)
counties_ppe.drop(columns=['districtname'], inplace=True)

ppe_languages = counties_ppe[["ppe", "county_names"]].groupby("county_names").mean()
ppe_languages['County'] = ppe_languages.index
ppe_languages = ppe_languages.merge(sub_language, left_on="county_names", right_on="County")
ppe_languages.set_index('County', inplace=True)

In [426]:
ppe_languages = ppe_languages.sort_values(by='ppe', ascending=False)
all_lang_colors = ['#636EFA', '#EF553B', "#00CC96", "#AB63FA", "#FFA15A"]
all_counties = []
for i in range(len(ppe_languages.index)):
    all_counties.append(str(i + 1) + ". " + ppe_languages.index[i])
all_lang_types = ppe_languages.iloc[:, 2:].columns
all_lang_labels = ['Spanish', 'Other Indo-European Languages', 'Asian and Pacific Island Languages', 'Other Languages']
fig = make_subplots(2, 1, subplot_titles=('Non-English Languages Spoken in WA Counties Ranked by School District Per Pupil Expenditure (PPE)',  
                                          ' WA Counties Ranked by School District Per Pupil Expenditure (PPE)'))

for i in range(len(all_lang_labels)):
    fig.add_trace(go.Bar(x=all_counties,
                    y=ppe_languages[all_lang_types[i]],
                    name=all_lang_labels[i],
                    marker_color=all_lang_colors[i]
                    ), row = 1, col = 1)
fig.add_trace(go.Bar(x=all_counties,
                y=ppe_languages['ppe'],
                name='PPE',
                marker_color=all_lang_colors[4]
                ), row = 2, col = 1)
fig.update_layout(autosize=False, width=1300, height=1400)
fig['layout']['xaxis']['title']='WA Counties'
fig['layout']['xaxis2']['title']='WA Counties'
fig['layout']['yaxis']['title']='Percentage of Language Type Spoken'
fig['layout']['yaxis2']['title']='Per Pupil Expenditure'

fig.show()

3. To what extent do Washington residents continue to practice their ancestral culture through language?
- We chose to first look at the demographic distribution as a percentage of people who identified as non-American. 
Then we ran a linear regression to measure how strong this relationship was, between the the diversity rate (non-American)of a county and the rate at which English is only used in households.

In [427]:
households = pd.read_csv("data/DP02AncestryLanguagesSpokenAtHome.csv")
households = reformat_census_df(households, 2)
households = clean_df(households)

In [428]:
# getting the total diversity for non-American ancestry

ancestry_backgrounds = households.iloc[:, -32:-4].copy().loc[(households.index.get_level_values("County"), "Estimate"), :]
ancestry_backgrounds = ancestry_backgrounds.apply(lambda x: x.str.replace(',','').astype(float))
ancestry_backgrounds.columns = [name.strip() for name in ancestry_backgrounds.columns]


diverse_count = []
full_count = [x.sum() for _, x in ancestry_backgrounds.iterrows()]
for _, row in ancestry_backgrounds.iloc[:, 1:].iterrows():
    diverse_count.append(row.sum())

ancestry_backgrounds['diversity_ratio'] = [x / y for x, y in zip(diverse_count, full_count)]

In [429]:
# getting the languages being used in each household
lang_usages = households.iloc[:, -45:-33].copy().loc[(households.index.get_level_values("County"), "Percent"), :]
ancestry_backgrounds.index = ancestry_backgrounds.index.droplevel(1)
lang_usages.index = lang_usages.index.droplevel(1)
diverse_lang = pd.DataFrame(ancestry_backgrounds['diversity_ratio']).merge(lang_usages, right_index=True, left_index=True)
diverse_lang.iloc[:, 2:] = diverse_lang.iloc[:, 2:].apply(lambda x: x.str.replace('%','').astype(float) / 100)

In [430]:
lang_counties = []
lang_counts = []
lang_kind = []

for idx, row in diverse_lang.iterrows():
    lang_counties.append(idx)
    lang_counts.append(row['english_only'])
    lang_kind.append('English Only')
    lang_counties.append(idx)
    lang_counts.append(row['language_other_than_english'])
    lang_kind.append('Language other than English')

lang_data = {}
lang_data['County'] = lang_counties
lang_data['Percent'] = lang_counts
lang_data['Language'] = lang_kind
lang_data = pd.DataFrame(lang_data).sort_values(by='Percent')
fig = px.bar(lang_data, x="Percent", y="County", color="Language", title="Diversity in Language in WA Counties", height=900)
fig.show()

/home/codespace/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [434]:
# let's test the strength of the relationships of the two variables: 
# diversity in each county and english only being spoken in households
fig = px.scatter(diverse_lang, x='diversity_ratio', y='english_only', trendline='ols', 
                    title='English only being spoken in Various Ancestrially Diverse Household by WA Counties',
                    labels=dict(diversity_ratio="Ancestrial Diversity", english_only="Households that Speak Only English"))
fig.show()

In [432]:
X = diverse_lang['english_only']
Y = diverse_lang['diversity_ratio']
X = sm.add_constant(X)
m = sm.OLS(Y.astype(float), X.astype(float))

r = m.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        diversity_ratio   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     179.6
Date:                Fri, 08 Mar 2024   Prob (F-statistic):           5.60e-16
Time:                        08:14:58   Log-Likelihood:                 97.342
No. Observations:                  40   AIC:                            -190.7
Df Residuals:                      38   BIC:                            -187.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0923      0.024      3.844      0.000       0.044       0.141
english_only     0.3741      0.028     13.402      0.000       0.318       0.431
==============================================================================
Omnibus:                        1.260   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.533   Jarque-Bera (JB):                1.089
Skew:                          -0.205   Prob(JB):                        0.580
Kurtosis:                       2.304   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""